<a href="https://colab.research.google.com/github/abhijeet1490/AI-Powered-Resume-and-Cover-Letter-Generator/blob/main/AI_Powered_Resume_and_Cover_Letter_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Project : AI Resume & Cover Letter Generator***
------------
*Submitted By -> Abhijeet Singh Generative AI Batch 4*

# Project Description
Title: AI-Powered Cover Letter & Resume Generator

## Objective:
- An interactive web app, built with Gradio for Google Colab, that uses Google's Gemini AI to automatically generate professional resumes and tailored cover letters. It transforms user input into polished, ready-to-use application documents.

### Key Features:

- Dual AI Generation: Creates both comprehensive resumes from structured forms and context-aware cover letters by analyzing the user's resume (PDF/text) against a job description.

- Interactive Web UI: Offers a simple, intuitive Gradio interface for a seamless user experience, requiring no coding knowledge.

- Direct Downloads: Instantly generates and provides download links for the final documents in both PDF and TXT formats.

### Use Cases:
- Ideal for any job applicant seeking a free, fast, and powerful tool to create high-quality, AI-enhanced application materials without needing specialized software.

# 1. Install all required packages

In [3]:
!pip install -q gradio langchain_google_genai PyPDF2 fpdf2 google-genai

# 2 .Import necessary libraries
 --- SETUP AND CONFIGURATION ---

Fetch API key from Colab's secret manager


In [8]:
# Import necessary libraries
import gradio as gr
import os
from PyPDF2 import PdfReader
from fpdf import FPDF
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
import time
import google.generativeai as genai

# --- SETUP AND CONFIGURATION ---

# Fetch API key from Colab's secret manager
try:
    GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("Google API key is empty.")
    genai.configure(api_key=GOOGLE_API_KEY)
except (userdata.SecretNotFoundError, ValueError) as e:
    raise ValueError("No Google API key found. Please set 'GOOGLE_API_KEY' in your Colab secrets (🔑).") from e

# Initialize Google Gemini LLM
gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.9,
    max_output_tokens=2048
)


## 3.Core Helper Functions

In [ ]:
def extract_text_from_pdf(pdf_file_path):
    try:
        with open(pdf_file_path, "rb") as file:
            pdf_reader = PdfReader(file)
            return "".join(page.extract_text() for page in pdf_reader.pages)
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return None

def create_pdf(text, filename="output.pdf"):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=11)
    encoded_text = text.encode('latin-1', 'replace').decode('latin-1')
    pdf.multi_cell(0, 5, encoded_text)

    output_dir = "outputs"
    os.makedirs(output_dir, exist_ok=True)

    unique_filename = f"{output_dir}/{filename.split('.')[0]}_{int(time.time())}.pdf"
    pdf.output(unique_filename)
    return unique_filename

def create_txt(text, filename="output.txt"):
    output_dir = "outputs"
    os.makedirs(output_dir, exist_ok=True)

    unique_filename = f"{output_dir}/{filename.split('.')[0]}_{int(time.time())}.txt"
    with open(unique_filename, "w", encoding="utf-8") as f:
        f.write(text)
    return unique_filename

# --- GRADIO PROCESSING FUNCTIONS ---

def process_cover_letter_request(res_format, res_file, pasted_resume, job_desc, user_name, company, manager, role, referral, ai_temp):
    if not all([job_desc, user_name, company, role]):
        return "Please fill in all required fields: Job Description, Your Name, Company, and Role.", None

    res_text = ""
    if res_format == 'Upload' and res_file is not None:
        res_text = extract_text_from_pdf(res_file.name)
        if res_text is None:
            return "Error: Could not read the uploaded PDF file.", None
    elif res_format == 'Paste':
        res_text = pasted_resume

    if not res_text:
        return "Error: Resume information is missing. Please upload or paste your resume.", None

    prompt = f"""
    Generate a professional and compelling cover letter based on the provided resume and job description.
    **My Resume Text:**\n{res_text}\n
    **The Job Description:**\n{job_desc}\n
    **Key Details for the Cover Letter:**
    - Candidate's Name: {user_name}
    - Job Title/Role Applied For: {role}
    - Company Name: {company}
    - Hiring Manager (if known): {manager}
    - How Candidate Heard About Opportunity: {referral}
    **Structure and Tone:**
    - The cover letter must have three content paragraphs.
    - **First Paragraph:** Introduce the candidate ({user_name}), state the position being applied for ({role}), mention where the opportunity was seen ({referral}), and briefly summarize their value proposition.
    - **Second Paragraph:** Create a strong connection between the candidate's experience (from the resume) and the qualifications listed in the job description. Highlight 2-3 key skills or accomplishments that are directly relevant.
    - **Third Paragraph:** Reiterate interest in the role and the company. Conclude with a call to action and thank the reader.
    - Maintain a professional and enthusiastic tone throughout.
    """
    try:
        gemini_llm.temperature = ai_temp
        response = gemini_llm.invoke(prompt)
        generated_text = response.content

        # Create both files and return their paths as a list
        pdf_path = create_pdf(generated_text, "cover_letter.pdf")
        txt_path = create_txt(generated_text, "cover_letter.txt")
        return generated_text, [pdf_path, txt_path]

    except Exception as e:
        return f"An error occurred while generating the cover letter: {e}", None

def process_resume_request(name, email, phone, linkedin, portfolio, summary, experience, education, skills, certifications, languages, interests):
    if not all([name, email, summary, experience, education, skills]):
        return "Please fill in all essential resume fields: Name, Email, Summary, Experience, Education, and Skills.", None

    resume_prompt = f"""
    Create a detailed, well-structured, and professional resume for {name}. Format it cleanly with clear headings.
    **1. Contact Information:**
    - Name: {name} | Email: {email} | Phone: {phone} | LinkedIn: {linkedin} | Portfolio: {portfolio}
    **2. Professional Summary:**\n{summary}\n
    **3. Work Experience:**\n(Parse and format the following professionally, using bullet points for responsibilities and achievements)\n{experience}\n
    **4. Education:**\n(Format the following academic qualifications)\n{education}\n
    **5. Skills:**\n(Organize the following skills into relevant categories if possible)\n{skills}\n
    **6. Certifications and Awards:**\n(List any provided certifications or awards)\n{certifications}\n
    **7. Languages:**\n(List languages and proficiency levels)\n{languages}\n
    **8. Interests/Hobbies:**\n(Include if provided, keep it brief and professional)\n{interests}\n
    """
    try:
        response = gemini_llm.invoke(resume_prompt)
        generated_text = response.content

        # Create both files and return their paths as a list
        pdf_path = create_pdf(generated_text, "resume.pdf")
        txt_path = create_txt(generated_text, "resume.txt")
        return generated_text, [pdf_path, txt_path]

    except Exception as e:
        return f"An error occurred while generating the resume: {e}", None

# --- GRADIO UI DEFINITION ---

with gr.Blocks(theme=gr.themes.Soft(), title="AI Document Generator") as demo:
    gr.Markdown("# 📝 AI-Powered Cover Letter and Resume Generator")
    gr.Markdown("Developed by Abhijeet Singh. Generate professional documents with AI assistance.")

    with gr.Tabs():
        with gr.TabItem("Cover Letter Generator"):
            gr.Markdown("### Step 1: Provide Your Information")
            with gr.Row():
                with gr.Column(scale=1):
                    user_name_cl = gr.Textbox(label="Your Full Name *")
                    company_cl = gr.Textbox(label="Company Name *")
                    role_cl = gr.Textbox(label="Job Title/Role You're Applying For *")
                    manager_cl = gr.Textbox(label="Hiring Manager (Optional)")
                    referral_cl = gr.Textbox(label="How did you hear about this opportunity?")
                    ai_temp_cl = gr.Slider(0.5, 1.0, value=0.9, step=0.1, label="AI Creativity (Temperature)")
                with gr.Column(scale=2):
                    job_desc_cl = gr.TextArea(label="Paste Job Description Here *", lines=10)

            gr.Markdown("### Step 2: Add Your Resume")
            res_format_cl = gr.Radio(["Upload", "Paste"], label="Do you want to upload or paste your resume?", value="Upload")
            res_file_cl = gr.File(label="📁 Upload your resume in PDF format", file_types=[".pdf"])
            pasted_resume_cl = gr.TextArea(label="Or Paste Resume Text Here", lines=15, visible=False)
            res_format_cl.change(lambda x: (gr.update(visible=x=='Upload'), gr.update(visible=x=='Paste')), res_format_cl, [res_file_cl, pasted_resume_cl])

            gr.Markdown("### Step 3: Generate!")
            cover_letter_btn = gr.Button("✨ Generate Cover Letter", variant="primary")

            gr.Markdown("---")
            gr.Markdown("## 📄 Generated Cover Letter")
            cover_letter_output = gr.Markdown(label="Output")

            #  Using a single component for multiple file downloads
            download_files_cl = gr.File(label="⬇️ Downloadable Files", file_count="multiple")

        with gr.TabItem("Resume Generator"):
            gr.Markdown("### Fill in your details below to generate a new resume.")
            with gr.Row():
                with gr.Column():
                    gr.Markdown("#### Personal Information")
                    name_res = gr.Textbox(label="Your Name *", placeholder="John Doe")
                    email_res = gr.Textbox(label="Email Address *", placeholder="john.doe@example.com")
                    phone_res = gr.Textbox(label="Phone Number", placeholder="+1234567890")
                    linkedin_res = gr.Textbox(label="LinkedIn Profile URL", placeholder="https://linkedin.com/in/johndoe")
                    portfolio_res = gr.Textbox(label="Portfolio URL (if applicable)", placeholder="https://yourportfolio.com")

                    gr.Markdown("#### Additional Information")
                    languages_res = gr.TextArea(label="Languages (Indicate proficiency)", lines=2)
                    interests_res = gr.TextArea(label="Interests/Hobbies (Optional)", lines=2)

                with gr.Column(scale=2):
                    gr.Markdown("#### Professional Details")
                    summary_res = gr.TextArea(label="Professional Summary *", lines=5, placeholder="A brief overview of your professional background...")
                    experience_res = gr.TextArea(label="Work Experience *", lines=10, placeholder="Detail your professional experience, starting with the most recent...")
                    education_res = gr.TextArea(label="Education *", lines=5, placeholder="List your academic qualifications, most recent first...")
                    skills_res = gr.TextArea(label="Skills *", lines=5, placeholder="List your professional skills (e.g., Python, Project Management, etc.)")
                    certifications_res = gr.TextArea(label="Certifications and Awards", lines=3)

            resume_btn = gr.Button("✨ Generate Resume", variant="primary")

            gr.Markdown("---")
            gr.Markdown("## 📄 Generated Resume")
            resume_output = gr.Markdown(label="Output")


            download_files_res = gr.File(label="⬇️ Downloadable Files", file_count="multiple")

    # --- Wire up button clicks to processing functions () ---
    cover_letter_btn.click(
        fn=process_cover_letter_request,
        inputs=[res_format_cl, res_file_cl, pasted_resume_cl, job_desc_cl, user_name_cl, company_cl, manager_cl, role_cl, referral_cl, ai_temp_cl],
        outputs=[cover_letter_output, download_files_cl] # FIXED
    )

    resume_btn.click(
        fn=process_resume_request,
        inputs=[name_res, email_res, phone_res, linkedin_res, portfolio_res, summary_res, experience_res, education_res, skills_res, certifications_res, languages_res, interests_res],
        outputs=[resume_output, download_files_res] # FIXED
    )

# Launch the Gradio App
demo.launch(debug=True)

/tmp/ipython-input-4122256626.py:110: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), title="AI Document Generator") as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d84b9ed7140e03633c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/tmp/ipython-input-4122256626.py:13: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=11)


# Challenges:

## Challenges and Solutions
1.✅Handling API Failures
Challenge: Invalid API keys or model names caused the app to crash.

Solution:
Added try-except blocks to catch API errors and display them in the UI instead of crashing. The code was updated to use a stable model name (gemini-1.5-flash-latest) to ensure reliability.


2.✅Simplifying User Input
Challenge: The interface for adding a resume (upload vs. paste) was cluttered.

Solution:
Used a Gradio radio button to dynamically show only the relevant input field (either the file uploader or the text area), creating a cleaner user experience.


3.✅Clean File Downloads
Challenge: Offering both PDF and TXT downloads resulted in large, confusing UI buttons.

Solution:

Switched to a single gr.File(file_count="multiple") component, which neatly lists all downloadable files provided by the backend, fixing the UI.


4.✅Parsing Inconsistent PDF Resume Formats
Challenge: Resumes uploaded as PDFs have varied layouts (e.g., multi-column, tables). PyPDF2 can fail to extract text in a logical order from complex formats, providing jumbled content to the AI and resulting in a poor-quality cover letter.

Solution:

The primary solution was to offer a "Paste Text" option as a reliable alternative. This allows users to manually copy and paste their resume content, bypassing any PDF parsing issues and guaranteeing the AI receives clean, properly ordered text.

5.✅Translating AI Formatting to PDF Output
Challenge: The Gemini model often generates text with markdown for formatting (like **Headings** or * Bullet points), but the fpdf2 library doesn't understand markdown and would just print it as plain text.

Solution:

A simple post-processing function was created to parse the AI's text output. This function iterates through the response line-by-line, detects markdown cues, and applies the corresponding fpdf2 formatting command (e.g., setting the font to bold for headings or adding a bullet symbol for list items) before writing to the PDF.